<a href="https://colab.research.google.com/github/camipower/BERT/blob/main/BERT_NER_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install huggingface transformers

In [1]:
!pip install transformers

     |████████████████████████████████| 2.3MB 5.1MB/s 
     |████████████████████████████████| 901kB 24.1MB/s 
     |████████████████████████████████| 3.3MB 42.4MB/s 


### Check GPU Availability

In [2]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [3]:
import tensorflow as tf
print(tf.version.VERSION)

2.5.0


### Install TensorFlow Model Garden

In [4]:
#!git clone --depth 1 -b v2.5.0 https://github.com/tensorflow/models.git

In [5]:
# install requirements to use tensorflow/models repository (restart runtime afterwards)
#!pip install -Uqr models/official/requirements.txt

### Import Dataset

In [4]:
import re
import numpy as np
import pandas as pd
from sklearn import preprocessing
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_hub as hub
import sys, os
#sys.path.append('models')
#from official.nlp.data import classifier_data_lib
#from official.nlp.bert import tokenization
#from official.nlp import optimization

from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig
#from transformers import BertTokenizerFast, TFBertModel, BertConfig

max_len = 384
configuration = BertConfig()

In [5]:
print("TF Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

TF Version:  2.5.0
Eager mode:  True
Hub version:  0.12.0
GPU is NOT AVAILABLE


### Setup Tokenizer

In [6]:
#tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
#tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
#tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

slow_tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

save_path = "bert_base_cased"
if not os.path.exists(save_path):
    os.makedirs(save_path)
slow_tokenizer.save_pretrained(save_path)

tokenizer = BertWordPieceTokenizer(save_path + "/vocab.txt", lowercase=False)

In [16]:
#print(tokenizer.wordpiece_tokenizer.tokenize(" ".join(sentences[0])))
#print(tokenizer.convert_tokens_to_ids(tokenizer.wordpiece_tokenizer.tokenize(" ".join(sentences[0]))))

#print(tokenizer.tokenize("I live in Germany."))
#print(tokenizer.convert_tokens_to_ids(tokenizer.tokenize("I live in Germany.")))

#slow_tokenizer.convert_ids_to_tokens(16)

## Encoding(num_tokens=17, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
print(tokenizer.encode("Alex lives with Abhishek in London and works for Microsoft.").tokens)

['O', 'B-per', 'O', 'O', 'B-org', 'B-geo', 'B-geo', 'B-geo', 'B-per', 'O', 'B-geo', 'O', 'O', 'O', 'B-org', 'O']

['[CLS]', 'Alex', 'lives', 'with', 'A', '##b', '##his', '##he', '##k', 'in', 'London', 'and', 'works', 'for', 'Microsoft', '.', '[SEP]']


In [19]:
!git clone https://github.com/camipower/BERT_NER_1.git

Cloning into 'BERT_NER_1'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 21 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (21/21), done.


In [20]:
data_csv = "BERT_NER_1/data/ner_dataset.csv"

df = pd.read_csv(data_csv, sep=',', encoding='latin1').fillna(method='ffill')

df.shape

(1048575, 4)

In [21]:
df.head(10)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
5,Sentence: 1,through,IN,O
6,Sentence: 1,London,NNP,B-geo
7,Sentence: 1,to,TO,O
8,Sentence: 1,protest,VB,O
9,Sentence: 1,the,DT,O


In [22]:
import re
df['Sentence #'] = df['Sentence #'].apply(lambda x: re.sub(r"Sentence: ", "", x))
df.head(10)

,Sentence #,Word,POS,Tag
0,1,Thousands,NNS,O
1,1,of,IN,O
2,1,demonstrators,NNS,O
3,1,have,VBP,O
4,1,marched,VBN,O
5,1,through,IN,O
6,1,London,NNP,B-geo
7,1,to,TO,O
8,1,protest,VB,O
9,1,the,DT,O


In [13]:
print(df['Tag'].unique())
print()
print(df['Tag'].value_counts())

['O' 'B-geo' 'B-gpe' 'B-per' 'I-geo' 'B-org' 'I-org' 'B-tim' 'B-art'
 'I-art' 'I-per' 'I-gpe' 'I-tim' 'B-nat' 'B-eve' 'I-eve' 'I-nat']

O        887908
B-geo     37644
B-tim     20333
B-org     20143
I-per     17251
B-per     16990
I-org     16784
B-gpe     15870
I-geo      7414
I-tim      6528
B-art       402
B-eve       308
I-art       297
I-eve       253
B-nat       201
I-gpe       198
I-nat        51
Name: Tag, dtype: int64


### Parse Data

In [14]:
#class SentenceGetter(object):
#
#    def __init__(self, data):
#        self.n_sent = 1
#        self.data = data
#        self.empty = False
#        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(),
#                                                           s['POS'].values.tolist(),
#                                                           s['Tag'].values.tolist())]
#        self.grouped = self.data.groupby('Sentence #').apply(agg_func)
#        self.sentences = [s for s in self.grouped]
#
#    def get_next(self):
#        try:
#            s = self.grouped[n_sent]
#            self.n_sent += 1
#            return s
#        except:
#            return None


In [15]:
# Get full document data structure
#getter = SentenceGetter(df)

In [18]:
# Get Sentence data
#sentences = [[s[0] for s in sent] for sent in getter.sentences]
#print(sentences[0])
#
# Get POS data
#pos = [[s[1] for s in sent] for sent in getter.sentences]
#print(pos[0])
#
# Get Tag data
#tag = [[s[2] for s in sent] for sent in getter.sentences]
#print(tag[0])

['Thousands', 'of', 'demonstrators', 'have', 'marched', 'through', 'London', 'to', 'protest', 'the', 'war', 'in', 'Iraq', 'and', 'demand', 'the', 'withdrawal', 'of', 'British', 'troops', 'from', 'that', 'country', '.']
['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP', 'TO', 'VB', 'DT', 'NN', 'IN', 'NNP', 'CC', 'VB', 'DT', 'NN', 'IN', 'JJ', 'NNS', 'IN', 'DT', 'NN', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O']


In [48]:
# Set a dictionary for mapping tag name to id
# tag2idx = {t: i for i, t in enumerate(tags_vals)}

# Recommend to set it by manual define, good for reusing

#tag2idx = {'B-art': 0,
#           'I-art': 1,
#           'B-eve': 2,
#           'I-eve': 3,
#           'B-geo': 4,
#           'I-geo': 5,
#           'B-gpe': 6,
#           'I-gpe': 7,
#           'B-nat': 8,
#           'I-nat': 9,
#           'B-org': 10,
#           'I-org': 11,
#           'B-per': 12,
#           'I-per': 13,
#           'B-tim': 14,
#           'I-tim': 15,
#           'X': 16,
#           'O': 17,
#           '[CLS]':18,
#           '[SEP]':19}

# Map also id to 

#idx2tag = {tag2idx[key]: key for key in tag2idx}
#
#print(tag2idx)
#print(idx2tag)
#

"# Set a dictionary for mapping tag name to id\n# tag2idx = {t: i for i, t in enumerate(tags_vals)}\n\n# Recommend to set it by manual define, good for reusing\n\ntag2idx = {'B-art': 0,\n           'I-art': 1,\n           'B-eve': 2,\n           'I-eve': 3,\n           'B-geo': 4,\n           'I-geo': 5,\n           'B-gpe': 6,\n           'I-gpe': 7,\n           'B-nat': 8,\n           'I-nat': 9,\n           'B-org': 10,\n           'I-org': 11,\n           'B-per': 12,\n           'I-per': 13,\n           'B-tim': 14,\n           'I-tim': 15,\n           'X': 16,\n           'O': 17,\n           '[CLS]':18,\n           '[SEP]':19}\n\n# Map also id to \n\nidx2tag = {tag2idx[key]: key for key in tag2idx}\n\nprint(tag2idx)\nprint(idx2tag)\n"

### Pre-process Dataset

In [23]:
def process_csv(data_csv):
    df = pd.read_csv(data_csv, sep=',', encoding='latin1')
    df.loc[:, 'Sentence #'] = df['Sentence #'].fillna(method='ffill')
    df['Sentence #'] = df['Sentence #'].apply(lambda x: re.sub(r"Sentence: ", "", x))
    tag_encoder = preprocessing.LabelEncoder()
    df.loc[:, 'Tag'] = tag_encoder.fit_transform(df["Tag"])
    #getter = SentenceGetter(df)
    #sentences = [[s[0] for s in sent] for sent in getter.sentences]
    ##pos = [[s[1] for s in sent] for sent in getter.sentences]
    #tags = [[s[2] for s in sent] for sent in getter.sentences]
    sentences = df.groupby("Sentence #")["Word"].apply(list).values
    tags = df.groupby("Sentence #")["Tag"].apply(list).values
    return sentences, tags, tag_encoder    # , pos

def create_inputs_targets(data_csv):
    
    dataset_dict = {
        'input_ids': [],
        'token_type_ids': [],
        'attention_mask': [],
        'tags': []
    }

    sentences, tags, tag_encoder = process_csv(data_csv)

    for sentence, tag in zip(sentences, tags):
        input_ids = []
        target_tags = []
        for i, word in enumerate(sentence):
            ids = tokenizer.encode(word, add_special_tokens=False)
            input_ids.extend(ids.ids)
            
            ## this is   a    differentiator -> ['this', 'is', 'a', 'different', '##ia', '##tor']
            ## tag1 tag2 tag3 tag4           -> [tag1,   tag2, tag3, tag4,       tag4,   tag4]
            num_tokens = len(ids)
            target_tags.extend([tag[i]] * num_tokens)
            #target_pos.extend([pos[i]] * num_tokens)   # if we want pos too (more needs to be done)
        
        # Pad truncate
        # - 2 because 2 special tokens need to be added ([CLS], [SEP])
        input_ids = input_ids[:max_len - 2]
        target_tags = target_tags[:max_len - 2]

        input_ids = [101] + input_ids + [102]   # Add special tokens ([CLS] -> 101, [SEP] -> 102)
        #target_tags = [16] + target_tags + [16]
        target_tags = [0] + target_tags + [0]
        token_type_ids = [0] * len(input_ids)
        attention_mask = [1] * len(input_ids)
        padding_len = max_len - len(input_ids)

        input_ids = input_ids + ([0] * padding_len)
        attention_mask = attention_mask + ([0] * padding_len)
        token_type_ids = token_type_ids + ([0] * padding_len)
        #target_tags = target_tags + ([17] * padding_len)
        target_tags = target_tags + ([0] * padding_len)

        dataset_dict['input_ids'].append(input_ids)
        dataset_dict['token_type_ids'].append(token_type_ids)
        dataset_dict['attention_mask'].append(attention_mask)
        dataset_dict['tags'].append(target_tags)
        assert len(target_tags) == max_len, f'{len(input_ids)}, {len(target_tags)}'

    for key in dataset_dict:
        dataset_dict[key] = np.array(dataset_dict[key])

    x = [
         dataset_dict['input_ids'],
         dataset_dict['token_type_ids'],
         dataset_dict['attention_mask']
    ]

    y = dataset_dict['tags']

    return x, y, tag_encoder





In [35]:
x, y, t_enc = create_inputs_targets(data_csv)

In [37]:
y[0]

array([16, 16, 16, 16, 16, 16, 16, 16, 16,  2, 16, 16, 16, 16, 16,  2, 16,
       16, 16, 16, 16,  3, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17,
       17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
       17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
       17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
       17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
       17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
       17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
       17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
       17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
       17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
       17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
       17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
       17, 17, 17, 17, 17

### Define model

In [24]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=False, reduction=tf.keras.losses.Reduction.NONE
)

def masked_ce_loss(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 17))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

def create_model(num_tags):
    # BERT encoder
    encoder = TFBertModel.from_pretrained('bert-base-cased')

    # NER model
    input_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    token_type_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    attention_mask = layers.Input(shape=(max_len,), dtype=tf.int32)
    embedding = encoder(
        input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask
    )[0]
    embedding = layers.Dropout(0.3)(embedding)
    tag_logits = layers.Dense(num_tags + 1, activation='softmax')(embedding)

    model = keras.Model(
        inputs=[input_ids, token_type_ids, attention_mask],
        outputs=[tag_logits],
    )

    optimizer = keras.optimizers.Adam(lr=3e-5)
    
    model.compile(optimizer=optimizer, loss=masked_ce_loss, metrics=['accuracy'])

    return model


### Create model

In [25]:
num_tags = pd.read_csv(data_csv, encoding='latin1')['Tag'].unique()

use_tpu = None
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    use_tpu = True
except:
    use_tpu = False

if use_tpu:
    # Create distribution strategy
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)

    # Create model
    with strategy.scope():
        model = create_model(len(num_tags))

else:
    model = create_model(len(num_tags))

model.summary()

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: grpc://10.17.208.122:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.17.208.122:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 384)]        0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 384)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 384)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     TFBaseModelOutputWit 108310272   input_1[0][0]                    
                                                                 input_3[0][0]                

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


### Train

In [26]:
x_train, y_train, tag_encoder = create_inputs_targets(data_csv)

BATCH_SIZE = 64 if use_tpu else 16

model.fit(
    x_train,
    y_train,
    epochs=3,
    verbose=1,
    batch_size=BATCH_SIZE,
    validation_split=0.1
)

Epoch 1/3


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond_8/Identity_3:0' shape=(None, 384) dtype=int64>]


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond_8/Identity_3:0' shape=(None, 384) dtype=int64>]


675/675 [==============================] - ETA: 0s - loss: nan - accuracy: 0.9915   

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond_8/Identity_3:0' shape=(None, 384) dtype=int64>]


675/675 [==============================] - 271s 287ms/step - loss: nan - accuracy: 0.9915 - val_loss: 0.0100 - val_accuracy: 0.9972
Epoch 2/3
675/675 [==============================] - 138s 204ms/step - loss: nan - accuracy: 0.9971 - val_loss: 0.0089 - val_accuracy: 0.9973
Epoch 3/3
675/675 [==============================] - 138s 205ms/step - loss: nan - accuracy: 0.9975 - val_loss: 0.0083 - val_accuracy: 0.9974


### Tag entities

In [92]:
def create_test_input_from_text(texts):
    dataset_dict = {
        "input_ids": [],
        "token_type_ids": [],
        "attention_mask": []
    }
    for sentence in texts:
        input_ids = []
        for idx, word in enumerate(sentence.split()):
            ids = tokenizer.encode(word, add_special_tokens=False)
            input_ids.extend(ids.ids)
            num_tokens = len(ids)
            
        # Pad and create attention masks.
        # Skip if truncation is needed
        input_ids = input_ids[:max_len - 2]

        input_ids = [101] + input_ids + [102]
        n_tokens = len(input_ids)
        token_type_ids = [0] * len(input_ids)
        attention_mask = [1] * len(input_ids)
        padding_len = max_len - len(input_ids)

        input_ids = input_ids + ([0] * padding_len)
        attention_mask = attention_mask + ([0] * padding_len)
        token_type_ids = token_type_ids + ([0] * padding_len)
        
        dataset_dict["input_ids"].append(input_ids)
        dataset_dict["token_type_ids"].append(token_type_ids)
        dataset_dict["attention_mask"].append(attention_mask)
        
    for key in dataset_dict:
        dataset_dict[key] = np.array(dataset_dict[key])

    x = [
        dataset_dict["input_ids"],
        dataset_dict["token_type_ids"],
        dataset_dict["attention_mask"],
    ]
    return x, n_tokens



def tag_named_entities(query):

    x_test, n_tokens = create_test_input_from_text(query)

    pred_test = model.predict(x_test)
    pred_tags = np.argmax(pred_test,2)[0][1:n_tokens-1]  # ignore predictions of padding tokens

    # create dictionary of tags and their indexes
    le_dict = dict(zip(tag_encoder.transform(tag_encoder.classes_), tag_encoder.classes_))
    predicted_tags = [le_dict.get(_, '[pad]') for _ in pred_tags]


    ids = x_test[0][0][:n_tokens]

    tokenizer_tokens = tokenizer.encode(tokenizer.decode(ids)).tokens

    # for debugging
    print(tokenizer_tokens)

    new_tokens, new_labels = [], []
    for token, label in zip(tokenizer_tokens[1:-1], predicted_tags):
        if token.startswith('##'):
            new_tokens[-1] = new_tokens[-1] + token[2:]
        else:
            new_tokens.append(token)
            new_labels.append(label)
    
    for token, label in zip(new_tokens, new_labels):
        print("{}\t{}".format(token, label))

    
    
    

In [93]:
tag_named_entities(["Alex lives with Abhishek in London and works for Microsoft Corp."])


['[CLS]', 'Alex', 'lives', 'with', 'A', '##b', '##his', '##he', '##k', 'in', 'London', 'and', 'works', 'for', 'Microsoft', 'Corp', '.', '[SEP]']
Alex	B-per
lives	O
with	O
Abhishek	B-per
in	O
London	B-geo
and	O
works	O
for	O
Microsoft	B-org
Corp	I-org
.	O


In [94]:
tag_named_entities(["Alex lives with Abhishek in London and works for Ayfie."])

['[CLS]', 'Alex', 'lives', 'with', 'A', '##b', '##his', '##he', '##k', 'in', 'London', 'and', 'works', 'for', 'A', '##y', '##fie', '.', '[SEP]']
Alex	B-per
lives	O
with	O
Abhishek	B-per
in	O
London	B-geo
and	O
works	O
for	O
Ayfie	B-per
.	O


In [95]:
tag_named_entities(["Alex lives with Abhishek in London and works in Ayfie."])

['[CLS]', 'Alex', 'lives', 'with', 'A', '##b', '##his', '##he', '##k', 'in', 'London', 'and', 'works', 'in', 'A', '##y', '##fie', '.', '[SEP]']
Alex	B-per
lives	O
with	O
Abhishek	B-per
in	O
London	B-geo
and	O
works	O
in	O
Ayfie	B-geo
.	O


In [96]:
tag_named_entities(["Alex lives with Abhishek in London and works for Ayfie Inc."])

['[CLS]', 'Alex', 'lives', 'with', 'A', '##b', '##his', '##he', '##k', 'in', 'London', 'and', 'works', 'for', 'A', '##y', '##fie', 'Inc', '.', '[SEP]']
Alex	B-per
lives	O
with	O
Abhishek	B-per
in	O
London	B-geo
and	O
works	O
for	O
Ayfie	B-org
Inc	I-org
.	O


In [49]:
print(tokenizer.encode("Alex lives with Abhishek in London and works for Microsoft.").tokens)
print(tokenizer.encode("Alex lives with Abhishek in London and works for Microsoft.").ids)

['[CLS]', 'Alex', 'lives', 'with', 'A', '##b', '##his', '##he', '##k', 'in', 'London', 'and', 'works', 'for', 'Microsoft', '.', '[SEP]']
[101, 3230, 2491, 1114, 138, 1830, 27516, 4638, 1377, 1107, 1498, 1105, 1759, 1111, 6998, 119, 102]
